# BCI command stream

In [1]:
import os
import time
import mne
import numpy as np
import pandas as pd

import bsl
from bsl import StreamPlayer, datasets
# from bsl.externals import pylsl  # distributed version of pylsl
from bsl.triggers import TriggerDef

import pylsl

import math
import matplotlib
import matplotlib.pyplot as plt
from pythonosc.udp_client import SimpleUDPClient

## Check LSL channels

In [2]:
streams = [stream.name() for stream in pylsl.resolve_streams()]
print (streams)

[]


## Analyse signal and send commands

In [ ]:
# Stream receiver initialization
receiver = bsl.StreamReceiver(bufsize=10, winsize=10, stream_name='igeb')

# MNE info and lighter info (without problematic T7) initialization
ch_names=['E1']
ch_types=['eeg']
info = mne.create_info(sfreq=250, ch_names=ch_names, ch_types=ch_types)

selected_channels = [True]
extended_selected_channels = [False]+selected_channels

# OSC client initialization
ip = "127.0.0.1"
port = 4545
client = SimpleUDPClient(ip, port)

# Wait bufsize seconds before launching
time.sleep(11)
print('Starting recording reference score')
k=0
# 10s to compute the reference
reference_scores = []
main_loop_scores = []
while k<40:
    start = time.time()
    receiver.acquire()
    signal_array, timestamps_array = receiver.get_window(stream_name='igeb')
    signal_array = np.nan_to_num(signal_array)
    raw = mne.io.RawArray(data=signal_array[:,extended_selected_channels].T, info=info)
    # Filter on the 10s
    raw.filter(1,30)
    # Analyzing the last half a second
    # To change that : crop([tmin, tmax, include_tmax, verbose])
    # For instance to get signal between 8.5 and 9.5s : raw.crop(tmin=8.5, tmax=9.5)
    raw.crop(tmin=9)
    psds, freqs = mne.time_frequency.psd_welch(raw, fmin=8, fmax=12, n_fft=125)  # 125
    score = np.mean(psds)
    reference_scores.append(score)
    
    k+=1
    end = time.time()
    time.sleep(0.25-(end - start)-0.0025)

ref_mean_score = np.mean(reference_scores)
ref_std_score = np.std(reference_scores)
print(f"Reference score : {reference_scores}")


# Main loop
while True:  # Could be True or k<k_max
    receiver.acquire()
    signal_array, timestamps_array = receiver.get_window(stream_name='igeb')
    signal_array = np.nan_to_num(signal_array)
    raw = mne.io.RawArray(data=signal_array[:,extended_selected_channels].T, info=info)
    # Filter on the 10s
    raw.filter(1,30)  # Bandpass between 1Hz and 30Hz.
    # Analyzing the last half a second
    raw.crop(tmin=9)
    psds, freqs = mne.time_frequency.psd_welch(raw, fmin=8, fmax=12, n_fft=125)  # 125
    score = np.mean(psds)
    
    score = (score-ref_mean_score)/ref_std_score/2  # TODO : change number 3 ?
#     print(score)
    main_loop_scores.append(score)
    
    client.send_message("/in/alpha", score)
    